## Activate virtual environment first through venv/Scripts/activate

In [26]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

print(sentiment_analysis("I love this!"))

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [1]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [2]:
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

In [4]:
import tqdm

data = pd.read_csv("bbc-text-1.csv")
tech = data[data["category"] == "tech"]
df1 = pd.DataFrame(tech)
pred_texts = list(df1["text"][0:5])

tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [5]:
predictions = trainer.predict(pred_dataset)

In [6]:
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [8]:
df = pd.DataFrame(list(zip(pred_texts,preds,labels,scores)), columns=['text','pred','label','score'])
df.head()
df["text"][3]

'halo 2 heralds traffic explosion the growing popularity of online gaming could spell problems for net service firms  warns network monitoring company sandvine.  it issued the warning following analysis which shows that traffic on the xbox game network increased fourfold on the launch day of halo 2. the 9 november traffic explosion has continued into december  said sandvine. service providers now need to make sure that their networks can cope with the increasing demands for bandwidth. as well as being a popular single-player title  halo 2 can be connected to microsoft s subscription-based broadband network  xbox live. gamers who want to play online can create their own clan  or team  and take on others to see how well they compare.  but the surge in numbers and huge demands for bandwidth should be a wake-up call to the industry which must ensure that their networks can cope with the increases in traffic  said sandvine s chief technology officer marc morin.  in a bid to cope and ease co